In [26]:
import igraph as ig
import json
import requests
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
import plotly.express as px
from collections import defaultdict
from word import Word 
from wordsdict import WordsDict

In [27]:
wordsList = ["repent", "atone", "amend", "insatiable"]
vocabDict = WordsDict(wordsList, "Vocabularies")
vocabDict.addWordStrs(["a", "b", "C"])
vocabDict.addWordStrs(["d", "atone", "a"])
vocabDict.printWordsDict()


vocabDict.addEdges([("repent", "atone"), ("repent", "C"), ("repent", "a"), ("repent", "amend"), ("b", "insatiable"), ("b", "b")])
vocabDict.printEdges()


vocabDict.addEdges([("repent", "b"), ("repent", "a"), ("C", "a"), ("atone", "amend"), ("a", "insatiable"), ("C", "b"), ("C", "a")], "uncategorized edges")
vocabDict.printAllEdges()

vocabDict.removeEdges([("C", "b"), ("atone", "b"), ("repent", "a")], "uncategorized edges")
vocabDict.printAllEdges()

Error addWordStr: Vocabularies validationCheck, 'atone' exist status == True!
Error addWordStr: Vocabularies validationCheck, 'a' exist status == True!
Printing words for Vocabularies:
['repent', 'atone', 'amend', 'insatiable', 'a', 'b', 'C', 'd'] 

Error addEdge: Vocabularies validationCheck, 'b' is dealing with itself!
Printing synonyms of Vocabularies
For repent's synonyms neighbors:
     ['atone', 'C', 'a', 'amend']
For atone's synonyms neighbors:
     ['repent']
For amend's synonyms neighbors:
     ['repent']
For insatiable's synonyms neighbors:
     ['b']
For a's synonyms neighbors:
     ['repent']
For b's synonyms neighbors:
     ['insatiable']
For C's synonyms neighbors:
     ['repent']


Error: C tried to add 'uncategorized edges' edge to a, but the edge already exists!
Error: a tried to add 'uncategorized edges' edge to C, but the edge already exists!
Printing neighbors for repent:
For repent's synonyms neighbors:
     ['atone', 'C', 'a', 'amend']
For repent's uncategorized e